In [43]:
import requests
import time
import os

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup

import pandas as pd

In [44]:
# Create Chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless") # This site cannot be scraped headless.
driver = webdriver.Chrome(options=chrome_options)

# Province list to iterate through to make it less resource intense.
prov_list = {1: "ON", 2: "BC", 3: "MB",
             4: "AB", 5: "NB", 6: "NL",
             7: "NT", 8: "NS", 9: "NU", 
             10: "PE", 11: "QC", 12: "SK", 
             13: "YT"
            }

# For testing.
prov = prov_list[7]
print(prov)

NT


In [45]:
driver.get(f"https://www.jobbank.gc.ca/jobsearch/jobsearch?sort=M&fprov={prov}")

In [ ]:
wait = WebDriverWait(driver, 2)
click_count = 0
button_present = driver.find_element(By.ID, 'moreresultbutton').is_displayed()
print(button_present)  
# Click the "more results", button to load all the listings to the page.
while button_present:
    try:
        print('Click attempt')
        button = wait.until(EC.element_to_be_clickable((By.ID, "moreresultbutton")))
        button.click()
        time.sleep(2)
        click_count += 1
        print(f'clicked: {click_count} times.')
        button_present = driver.find_element(By.ID, 'moreresultbutton').is_displayed()
    except:
        print("All done!")
        break

In [47]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
main_box = soup.findAll("article") # Find all the job listings on the page.
len(main_box)

177

In [48]:
data_list = [] # Store job listing data for further processing

# Iterate through all job listings.
for box in main_box:
    temp_dict = {} # Dict of job posting.
    temp_dict['job_title'] = box.find("span", {"class": "noctitle"}).text.replace('\n', '').replace('\t', '').strip()
    temp_dict['business'] =  box.find("li", {"class": "business"}).text
    temp_dict['location'] =  box.find("li", {"class": "location"}).text.replace('\n', '').replace('\t', '').strip()
    temp_dict['salary'] =    box.find("li", {"class": "salary"}).text.replace('\n', '').replace('\t', '').strip()
    temp_dict['link'] =      box.find("a", {"class": "resultJobItem"})["href"]
    temp_dict['date'] =      box.find("li", {"class": "date"}).text
    
    data_list.append(temp_dict)

In [49]:
# End selenium session.        
driver.close()

In [50]:
df = pd.DataFrame(data_list)
len(data_list)

177

In [51]:
compresion_opts = dict(method='zip', archive_name=f'out-{prov}.csv')
# make folder and output to it
os.makedirs('output', exist_ok=True)
df.to_csv(f'output/out-{prov}.zip', compression=compresion_opts)